In [4]:
import csv
import json  # Adicionando a importação do módulo json
import time
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [5]:
# Função para formatar o código com pontos
def format_code(parts):
    return f"{parts[0]}.{parts[1]}.{parts[2]}.{parts[3]}.{parts[4]}.{parts[5]}"

In [6]:
# Função para salvar o PDF com um nome específico
def save_pdf(driver, filename):
    # Configura o caminho para salvar o PDF
    save_path = os.path.join(os.getcwd(), filename)
    driver.execute_script('window.print();')
    time.sleep(30)  # Aguarda 30 segundos para a impressão ser concluída
    # Mover o PDF salvo para o caminho desejado
    # Nota: pode ser necessário ajustar o caminho dependendo de como o PDF é salvo
    if os.path.isfile('Save as PDF.pdf'):
        os.rename('Save as PDF.pdf', save_path)
    else:
        print(f"Arquivo PDF não encontrado. Verifique o nome do arquivo gerado.")

In [7]:
# Função para extrair o total do HTML
def extract_total(driver):
    try:
        total_element = driver.find_element(By.CSS_SELECTOR, '#totais .vencida')
        return total_element.text.strip()
    except Exception as e:
        print(f"Erro ao extrair o total: {e}")
        return "N/A"

In [8]:
# Configurando as opções do Chrome
options = ChromeOptions()
options.add_argument("--start-maximized")  # Inicia o Chrome em tela cheia
options.add_experimental_option("prefs", {
    "printing.print_preview_sticky_settings.appState": json.dumps({
        "recentDestinations": [{
            "id": "Save as PDF",
            "origin": "local",
            "account": "",
        }],
        "selectedDestinationId": "Save as PDF",
        "version": 2,
        "pageSize": "TABLOID",
        "scaling": "DEFAULT",
        "isHeaderFooterEnabled": False,
        "isBackgroundGraphicsEnabled": False,
    })
})
options.add_argument("--kiosk-printing")  # Permite impressão sem diálogo

# Inicializando o Chrome WebDriver com as opções configuradas
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

In [9]:
# Carregar o arquivo CSV original e criar o novo CSV com a coluna de totais
with open('dados.csv', newline='') as csvfile:
    csvreader = csv.reader(csvfile)
    rows = list(csvreader)  # Carregar todas as linhas do CSV original
    header = rows[0]  # Cabeçalho do CSV original
    data_rows = rows[1:]  # Dados do CSV original

    # Criar um novo arquivo CSV para resultados com totais
    with open('resultados.csv', mode='w', newline='', encoding='utf-8') as results_file:
        results_writer = csv.writer(results_file)
        results_writer.writerow(header + ['Total'])  # Adiciona o cabeçalho com a coluna 'Total'

        try:
            for row in data_rows:
                # Formatar o código com as partes
                code_base = format_code(row[:6])
                
                # Tentar com final .0 e .1
                for suffix in ['.0', '.1']:
                    code = f"{code_base}{suffix}"
                    
                    driver.get("https://e-gov.betha.com.br/cdweb/resource.faces?params=FzKTkvNNaxrljKCe99HGiFN24zbCAH8Y")
                    driver.implicitly_wait(5)

                    try:
                        # Interagir com os elementos da página
                        element = driver.find_element(By.CLASS_NAME, 'inscImobiliaria')
                        element.click()

                        element = driver.find_element(By.ID, 'mainForm:inscrImob')
                        element.send_keys(code)
                        element.send_keys(Keys.ENTER)
                        
                        # Esperar o carregamento dos resultados
                        WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.ID, 'totais'))
                        )

                        # Verificar se há alertas ou mensagens de erro
                        alert_present = False
                        try:
                            alert = driver.switch_to.alert
                            alert_present = True
                        except:
                            pass

                        if not alert_present:
                            # Se não houver erro, esperar 30 segundos e salvar o PDF
                            time.sleep(30)  # Esperar 30 segundos para garantir que o conteúdo esteja totalmente carregado
                            filename = f"resultado_{code}.pdf"
                            save_pdf(driver, filename)
                            total = extract_total(driver)
                            results_writer.writerow(row + [total])
                            print(f"Consulta bem-sucedida com código {code}. PDF salvo como {filename} e resultado salvo no CSV.")
                            break
                        else:
                            print(f"Erro detectado com código {code}. Tentando com {code_base}.1")
                    except Exception as e:
                        print(f"Erro na consulta com código {code}: {e}")

        finally:
            driver.quit()

Erro na consulta com código 00.05.85.24.210.000.0: Message: 
Stacktrace:
	GetHandleVerifier [0x0031C203+27395]
	(No symbol) [0x002B3E04]
	(No symbol) [0x001B1B7F]
	(No symbol) [0x001F2C65]
	(No symbol) [0x001F2D3B]
	(No symbol) [0x0022EC82]
	(No symbol) [0x002139E4]
	(No symbol) [0x0022CB24]
	(No symbol) [0x00213736]
	(No symbol) [0x001E7541]
	(No symbol) [0x001E80BD]
	GetHandleVerifier [0x005D3AB3+2876339]
	GetHandleVerifier [0x00627F7D+3221629]
	GetHandleVerifier [0x0039D674+556916]
	GetHandleVerifier [0x003A478C+585868]
	(No symbol) [0x002BCE44]
	(No symbol) [0x002B9858]
	(No symbol) [0x002B99F7]
	(No symbol) [0x002ABF4E]
	BaseThreadInitThunk [0x77087BA9+25]
	RtlInitializeExceptionChain [0x77AAC11B+107]
	RtlClearBits [0x77AAC09F+191]
	(No symbol) [0x00000000]

Erro na consulta com código 00.05.85.24.210.000.1: Message: 
Stacktrace:
	GetHandleVerifier [0x0031C203+27395]
	(No symbol) [0x002B3E04]
	(No symbol) [0x001B1B7F]
	(No symbol) [0x001F2C65]
	(No symbol) [0x001F2D3B]
	(No symbol

KeyboardInterrupt: 